In [1]:
using Pkg
using Revise
Pkg.activate(joinpath(@__DIR__, "../../"))
using Globtim
using DynamicPolynomials, DataFrames
using GLMakie
using StaticArrays
# Constants and Parameters
const n, a, b = 3, 12, 100 
const scale_factor = a / b   # Scaling factor appears in `main_computation`, maybe it should be a parameter.
const delta, alpha = 0.5, 1 / 10  # Sampling parameters
const tol_l2 = 3e-4            # Define the tolerance for the L2-norm
f = tref_3d # Objective function

  Activating project at `~/globtim`


tref_3d (generic function with 1 method)

We set the number of samples used to generate the approximant. It is annoying that the error goes up while the degree has increased.

In [ ]:
center = [0.0, 0.0, 0.0];
d = 6 # initial degree 
SMPL = 40 # Number of samples
TR = test_input(f, 
                dim = n,
                center=center,
                GN=SMPL, 
                sample_range=scale_factor, 
                degree_max =d+4
                )
pol_cheb = Constructor(TR, d, basis=:chebyshev)
pol_lege = Constructor(TR, d, basis=:legendre);

@polyvar(x[1:n]); # Define polynomial ring 

current L2-norm: 2.937296090301863

Solve the system of partial derivatives using `Homotopy_COntinuation.jl`. 

In [3]:
# df_cheb = solve_and_parse(pol_cheb, x, f, TR)
pts_cheb = solve_polynomial_system(x, TR.dim, d, pol_cheb.coeffs; basis=:chebyshev)
df_cheb = process_critical_points(pts_cheb, f, TR)
sort!(df_cheb, :z, rev=false)

Dimension m of the vector space: 84
Tracking 125 paths...   2%|▌                            |  ETA: 0:03:33



Tracking 125 paths... 100%|█████████████████████████████| Time: 0:00:03
  # paths tracked:                  125
  # non-singular solutions (real):  125 (43)
  # singular endpoints (real):      0 (0)
  # total solutions (real):         125 (43)

=== Starting Critical Points Processing (dimension: 3) ===
Processed 34 points (0.044s)


Row,x1,x2,x3,z
,Float64,Float64,Float64,Float64
1,0.0946767,-0.100803,0.0478126,-1.35471
2,0.0954837,-0.036258,0.0963559,-1.0512
3,0.0951802,-0.100564,0.086008,-0.513498
4,0.0946822,0.0364523,0.0500552,-0.189736
5,0.0959057,0.0347423,0.107578,-0.0902982
6,0.0946818,0.103736,0.0386501,0.0858311
7,0.0946734,-0.0402016,0.044146,0.177413
8,-0.033101,-0.0292586,-0.0576996,0.196693
9,0.0974327,0.0347166,-0.0680615,0.29749


In [10]:
grid = scale_factor * generate_grid(3, 100)  # 3D grid
println("grid")
println(typeof(grid))
println(size(grid))

grid
Array{SVector{3, Float64}, 3}
(101, 101, 101)


In [106]:
"""
    validate_points_and_get_ranges(points::Vector{SVector{3,T}}) where T<:AbstractFloat

Returns validated ranges for x, y, z coordinates. Handles NaN filtering and validation.
"""
function validate_points_and_get_ranges(points::Vector{SVector{3,T}}) where {T<:AbstractFloat}
    valid_points = filter(p -> !any(isnan, p), points)

    if isempty(valid_points)
        throw(ArgumentError("No valid points found in the input points"))
    end

    x_range = extrema(p[1] for p in valid_points)
    y_range = extrema(p[2] for p in valid_points)
    z_range = extrema(p[3] for p in valid_points)

    if any(isnan, x_range) || any(isnan, y_range) || any(isnan, z_range)
        throw(ArgumentError("Invalid ranges detected in the points"))
    end

    return x_range, y_range, z_range
end

"""
    setup_3d_axis(fig::Figure, ranges::Tuple{Tuple,Tuple,Tuple})

Creates and configures a 3D axis with given ranges.
"""
function setup_3d_axis(fig::Figure, ranges::Tuple{Tuple,Tuple,Tuple})
    x_range, y_range, z_range = ranges

    ax = Axis3(fig[1, 1],
        title="Level Set Visualization",
        xlabel="x₁",
        ylabel="x₂",
        zlabel="x₃")

    limits!(ax, x_range..., y_range..., z_range...)

    return ax
end

"""
    create_level_controls(fig::Figure, z_range::Tuple{T,T}) where T<:AbstractFloat

Creates slider and label for level control.
"""
function create_level_controls(fig::Figure, z_range::Tuple{T,T}) where {T<:AbstractFloat}
    z_min, z_max = z_range
    if isnan(z_min) || isnan(z_max)
        throw(ArgumentError("Invalid z_range provided"))
    end

    level_slider = Slider(fig[2, 1],
        range=range(z_min, z_max, length=1000),
        startvalue=z_min)

    level_label = Label(fig[3, 1],
        @lift(string("Level: ", round($(level_slider.value), digits=3))),
        tellwidth=false)

    return level_slider, level_label
end

"""
    compute_level_set_points(f, points::Vector{SVector{3,T}}, level::T, tolerance::T) where T<:AbstractFloat

Computes points for the level set visualization.
"""
function compute_level_set_points(f, points::Vector{SVector{3,T}}, level::T, tolerance::T) where {T<:AbstractFloat}
    values = Vector{T}(undef, length(points))

    for (i, point) in enumerate(points)
        if !any(isnan, point)
            values[i] = try
                f(point)
                println("Computed value: ", f(point))  # Debug print
            catch e
                @warn "Error computing function value" point = point exception = e
                NaN
            end
        else
            values[i] = NaN
        end
    end

    level_data = LevelSetData(points, values, level)
    return to_makie_format(level_data), values
end

"""
    filter_visible_points(formatted_data, values::Vector{T}, level::T, tolerance::T) where T<:AbstractFloat

Filters points that should be visible at the current level.
"""
function filter_visible_points(formatted_data, values::Vector{T}, level::T, tolerance::T) where {T<:AbstractFloat}
    new_points = Point3f[]
    if !isempty(formatted_data.xyz[1])
        for (i, (x, y, z)) in enumerate(zip(formatted_data.xyz...))
            if !any(isnan, (x, y, z)) && abs(values[i] - level) ≤ tolerance
                push!(new_points, Point3f(x, y, z))
            end
        end
    end
    return new_points
end

"""
    filter_visible_data_points(df::DataFrame, level::T, tolerance::T, TR) where T<:AbstractFloat

Filters data points that should be visible at the current level.
Transforms points from actual domain to [-1,1]^3 domain for consistent visualization.
"""
function filter_visible_data_points(df::DataFrame, level::T, tolerance::T, TR) where {T<:AbstractFloat}
    visible_points = Point3f[]
    println("Processing $(nrow(df)) data points")  # Debug print

    for row in eachrow(df)
        values = [row["x1"], row["x2"], row["x3"], row["z"]]
        if !any(isnan, values) && abs(row["z"] - level) ≤ tolerance
            scaled_point = @. 2 * ([row["x1"], row["x2"], row["x3"]] - TR.center) / TR.sample_range
            push!(visible_points, Point3f(scaled_point...))
            println("Added point: ", scaled_point)  # Debug print
        end
    end
    println("Found $(length(visible_points)) visible points")  # Debug print
    return visible_points
end

function create_interactive_level_set_view(;
    f,
    points::Vector{SVector{3,T}},
    df::DataFrame,
    z_range::Tuple{T,T},
    TR,
    params::VisualizationParameters{T}=VisualizationParameters{T}()
) where {T<:AbstractFloat}
    # ... (previous code remains the same)

    function update_visualization(level::T) where {T<:AbstractFloat}
        if isnan(level)
            @warn "Invalid level value"
            return
        end

        try
            formatted_data, values = compute_level_set_points(f, points, level, params.point_tolerance)
            level_points[] = filter_visible_points(formatted_data, values, level, params.point_tolerance)

            # Debug prints for data points
            println("Updating visualization for level: ", level)
            new_data_points = filter_visible_data_points(df, level, params.point_tolerance, TR)
            println("Number of data points to display: ", length(new_data_points))
            data_points[] = new_data_points
        catch e
            @warn "Error in visualization update" exception = e
            level_points[] = Point3f[]
            data_points[] = Point3f[]
        end
    end

    on(level_slider.value) do level
        update_visualization(level)
    end

    try
        update_visualization(z_range[1])
    catch e
        @warn "Error in initial visualization" exception = e
    end

    axislegend(ax, position=:rt)

    return fig
end

create_interactive_level_set_view (generic function with 1 method)

In [107]:
fig = create_level_set_visualization(f, grid, df_cheb, (-3.0, 6.))

In [84]:
"""
    generate_box_samples(
        center::Vector{T}, 
        box_ranges::Tuple{Tuple,Tuple,Tuple}, 
        GN::Int=5
    ) where T<:AbstractFloat

Generates a 3D array of sample points in a box around the center point.
Returns Array{SVector{3,T}, 3} of size (GN, GN, GN)
"""
function generate_box_samples(
    center::Vector{T},
    box_ranges::Tuple{Tuple,Tuple,Tuple},
    GN::Int=5
) where {T<:AbstractFloat}
    # Extract ranges
    (x_min, x_max), (y_min, y_max), (z_min, z_max) = box_ranges

    # Calculate box dimensions (1/20 of total range)
    x_range = (x_max - x_min) / 20
    y_range = (y_max - y_min) / 20
    z_range = (z_max - z_min) / 20

    # Initialize 3D array for samples
    samples = Array{SVector{3,T},3}(undef, GN, GN, GN)

    # Generate grid points
    for i in 1:GN, j in 1:GN, k in 1:GN
        # Convert grid indices to [-0.5, 0.5] range
        x_norm = (i - 1) / (GN - 1) - 0.5
        y_norm = (j - 1) / (GN - 1) - 0.5
        z_norm = (k - 1) / (GN - 1) - 0.5

        # Scale by box size and add to center
        x = center[1] + x_norm * x_range
        y = center[2] + y_norm * y_range
        z = center[3] + z_norm * z_range

        samples[i, j, k] = SVector{3,T}(x, y, z)
    end

    return samples
end

"""
    process_box_samples(
        samples::Array{SVector{3,T}, 3}, 
        f, 
        level::T, 
        tolerance::T
    ) where T<:AbstractFloat

Processes the 3D array of box samples similar to the original grid points.
"""
function process_box_samples(
    samples::Array{SVector{3,T},3},
    f,
    level::T,
    tolerance::T
) where {T<:AbstractFloat}
    # Convert 3D array to vector for processing
    samples_vec = vec(samples)
    values = Vector{T}(undef, length(samples_vec))

    for (i, point) in enumerate(samples_vec)
        values[i] = try
            f(point)
        catch e
            @warn "Error computing function value" point = point exception = e
            NaN
        end
    end

    level_data = LevelSetData(samples_vec, values, level)
    formatted_data, values = to_makie_format(level_data), values

    # Filter points near the level set
    near_level_points = Point3f[]
    if !isempty(formatted_data.xyz[1])
        for (i, (x, y, z)) in enumerate(zip(formatted_data.xyz...))
            if !any(isnan, (x, y, z)) && abs(values[i] - level) ≤ tolerance
                push!(near_level_points, Point3f(x, y, z))
            end
        end
    end

    return near_level_points
end

process_box_samples

In [103]:
function filter_visible_data_points(df::DataFrame, level::T, tolerance::T, TR) where {T<:AbstractFloat}
    visible_points = Point3f[]
    println("Processing $(nrow(df)) data points")  # Debug print

    for row in eachrow(df)
        values = [row["x1"], row["x2"], row["x3"], row["z"]]
        if !any(isnan, values) && abs(row["z"] - level) ≤ tolerance
            scaled_point = @. 2 * ([row["x1"], row["x2"], row["x3"]] - TR.center) / TR.sample_range
            push!(visible_points, Point3f(scaled_point...))
            println("Added point: ", scaled_point)  # Debug print
        end
    end
    println("Found $(length(visible_points)) visible points")  # Debug print
    return visible_points
end

filter_visible_data_points (generic function with 2 methods)

In [104]:
"""
    create_interactive_level_set_view(; f, points, df, z_range, TR, params)

Create an interactive visualization of level sets with data points and box samples.
"""
function create_interactive_level_set_view(;
    f,
    points::Vector{SVector{3,T}},
    df::DataFrame,
    z_range::Tuple{T,T},
    TR,
    params::VisualizationParameters{T}=VisualizationParameters{T}()
) where {T<:AbstractFloat}
    
    # Create figure and get ranges
    fig = Figure(size=params.fig_size)
    ranges = validate_points_and_get_ranges(points)
    ax = setup_3d_axis(fig, ranges)
    
    # Create level slider and label
    level_slider, level_label = create_level_controls(fig, z_range)
    
    # Create Observables
    level_points = Observable(Point3f[])
    data_points = Observable(Point3f[])
    box_sample_points = Observable(Point3f[])
    
    # Create scatter plots
    scatter!(ax, level_points,
        color=:blue,
        markersize=2,
        label="Level Set")
    
    scatter!(ax, data_points,
        color=:orange,
        marker=:diamond,
        markersize=20,
        label="Data Points")
    
    scatter!(ax, box_sample_points,
        color=:purple,
        markersize=1,
        label="Box Samples")
    
    function update_visualization(level::T) where {T<:AbstractFloat}
        if isnan(level)
            @warn "Invalid level value"
            return
        end

        try
            # Update level set points
            formatted_data, values = compute_level_set_points(f, points, level, params.point_tolerance)
            level_points[] = filter_visible_points(formatted_data, values, level, params.point_tolerance)
            
            # Update data points
            visible_points = filter_visible_data_points(df, level, params.point_tolerance, TR)
            data_points[] = visible_points
            
            # Generate and process box samples
            if !isempty(visible_points)
                all_box_samples = Point3f[]
                for point in visible_points
                    center = [point[1], point[2], point[3]]
                    box_samples = generate_box_samples(center, ranges, 5)  # GN=5
                    near_level_samples = process_box_samples(box_samples, f, level, params.point_tolerance)
                    append!(all_box_samples, near_level_samples)
                end
                box_sample_points[] = all_box_samples
            else
                box_sample_points[] = Point3f[]
            end
            
        catch e
            @warn "Error in visualization update" exception=e
            level_points[] = Point3f[]
            data_points[] = Point3f[]
            box_sample_points[] = Point3f[]
        end
    end

    on(level_slider.value) do level
        update_visualization(level)
    end

    try
        update_visualization(z_range[1])
    catch e
        @warn "Error in initial visualization" exception=e
    end

    axislegend(ax, position=:rt)
    return fig
end

create_interactive_level_set_view

In [105]:
params = VisualizationParameters(
    fig_size=(800, 800),  # or whatever size you prefer
    point_tolerance=1e-2   # adjust as needed
)

# Create the interactive visualization
fig = create_interactive_level_set_view(;
    f=f,                    # your function
    points=points,     # vectorize the grid points
    df=df_cheb,                  # your dataframe with original points
    z_range=(-3., 2.),        # your z range
    TR=TR,                  # your transform
    params=params           # optional parameters
)

# Display the figure
display(fig)

Processing 34 data points
Found 0 visible points


GLMakie.Screen(...)

In [74]:
points = vec(grid) 
augmented_points = create_augmented_samples(points, df_cheb, TR, verbose=true);
aug_smpl = vec(augmented_points)

Initial number of base points: 1030301
After transformation: 1030301
After adding base points: 1030301
Critical point 1: Added 125 new points
Critical point 2: Added 125 new points
Critical point 3: Added 125 new points
Critical point 4: Added 125 new points
Critical point 5: Added 125 new points
Critical point 6: Added 125 new points
Critical point 7: Added 125 new points
Critical point 8: Added 125 new points
Critical point 9: Added 125 new points
Critical point 10: Added 125 new points
Critical point 11: Added 125 new points
Critical point 12: Added 125 new points
Critical point 13: Added 125 new points
Critical point 14: Added 125 new points
Critical point 15: Added 125 new points
Critical point 16: Added 125 new points
Critical point 17: Added 125 new points
Critical point 18: Added 125 new points
Critical point 19: Added 125 new points
Critical point 20: Added 125 new points
Critical point 21: Added 125 new points
Critical point 22: Added 125 new points
Critical point 23: Added 1

1034551-element Vector{SVector{3, Float64}}:
 [0.007199129255473076, 0.007199129255473076, 0.007199129255473076]
 [0.0071921645628667975, 0.007199129255473076, 0.007199129255473076]
 [0.00717824191554453, 0.007199129255473076, 0.007199129255473076]
 [0.007157374782768378, 0.007199129255473076, 0.007199129255473076]
 [0.00712958335214162, 0.007199129255473076, 0.007199129255473076]
 [0.007094894510078505, 0.007199129255473076, 0.007199129255473076]
 [0.007053341815793387, 0.007199129255473076, 0.007199129255473076]
 [0.007004965468834349, 0.007199129255473076, 0.007199129255473076]
 [0.006949812270192736, 0.007199129255473076, 0.007199129255473076]
 [0.0068879355770262215, 0.007199129255473076, 0.007199129255473076]
 ⋮
 [-0.09915296013497087, 0.09869673222782906, -0.03551587243191711]
 [-0.09615296013497086, 0.09869673222782906, -0.03551587243191711]
 [-0.09315296013497086, 0.09869673222782906, -0.03551587243191711]
 [-0.09015296013497086, 0.09869673222782906, -0.03551587243191711]
 [-0

In [81]:
fig = create_interactive_level_set_view(
    f=tref_3d,
    points=points,  # Already in [-1,1]^3
    df=df_cheb,                    # In actual domain
    z_range=(-3.0, 2.0),
    TR=TR,                    # Added TR for scaling
    params=VisualizationParameters()
)
display(fig)

GLMakie.Screen(...)

In [80]:
grids = create_augmented_samples(base_points, df_cheb, TR)
fig2 = create_interactive_level_set_view(
    f=tref_3d,
    points=points,  # All points in [-1,1]^3
    df=df_cheb,                    # In actual domain
    z_range=(-3.0, 2.0),
    TR=TR,
    params=VisualizationParameters()
)
display(fig2)

UndefVarError: UndefVarError: `base_points` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [78]:
struct SamplingGrids
    base_points::Vector{SVector{3,Float64}}  # In [-1,1]^3
    refined_grids::Vector{Vector{SVector{3,Float64}}}  # In [-1,1]^3
    grid_centers::Vector{SVector{3,Float64}}  # In actual domain
end

function create_augmented_samples(
    base_points::Vector{SVector{3,Float64}},  # Already in [-1,1]^3
    df_cheb::DataFrame,  # In actual domain
    TR;
    N_sub::Int=5,
    cube_side_ratio::Float64=0.1,
    verbose::Bool=true
)::SamplingGrids
    println("Initial number of base points: ", length(base_points))

    # Initialize containers for refined grids and their centers
    refined_grids = Vector{Vector{SVector{3,Float64}}}()
    grid_centers = Vector{SVector{3,Float64}}()

    # Create refined grid around each critical point
    for (i, row) in enumerate(eachrow(df_cheb))
        # Store the actual domain center point
        center_point = SVector{3,Float64}(row.x1, row.x2, row.x3)
        push!(grid_centers, center_point)

        # Create grid in actual domain and transform back to [-1,1]^3
        grid_points = create_refined_grid(
            center_point,
            TR,
            N_sub,
            cube_side_ratio
        )

        # Transform grid points back to [-1,1]^3 domain
        normalized_grid = transform_points_to_unit_cube(grid_points, TR)
        push!(refined_grids, normalized_grid)

        if verbose
            println("Critical point $i: Created grid with $(length(grid_points)) points")
        end
    end

    return SamplingGrids(base_points, refined_grids, grid_centers)
end

function create_refined_grid(
    center::SVector{3,Float64},  # In actual domain
    TR,
    N_sub::Int,
    cube_side_ratio::Float64
)::Vector{SVector{3,Float64}}
    side_length = TR.sample_range * cube_side_ratio
    half_side = side_length / 2

    x_coords = range(center[1] - half_side, center[1] + half_side, length=N_sub)
    y_coords = range(center[2] - half_side, center[2] + half_side, length=N_sub)
    z_coords = range(center[3] - half_side, center[3] + half_side, length=N_sub)

    return vec([SVector{3,Float64}(x, y, z)
                for x in x_coords,
                y in y_coords,
                z in z_coords])
end

function transform_points_to_unit_cube(
    points::Vector{SVector{3,Float64}},  # In actual domain
    TR
)::Vector{SVector{3,Float64}}
    return map(points) do point
        SVector{3,Float64}(
            2.0 * (point[1] - TR.center[1]) / TR.sample_range,
            2.0 * (point[2] - TR.center[2]) / TR.sample_range,
            2.0 * (point[3] - TR.center[3]) / TR.sample_range
        )
    end
end

# Function to get all points in [-1,1]^3 format for visualization
function get_all_points(grids::SamplingGrids)::Vector{SVector{3,Float64}}
    all_points = copy(grids.base_points)
    for grid in grids.refined_grids
        append!(all_points, grid)
    end
    return all_points
end



get_all_points (generic function with 1 method)

In [76]:
function create_augmented_samples(
    base_points::Vector{SVector{3,Float64}},
    df_cheb::DataFrame,
    TR;
    N_sub::Int=5,
    grid_size::Float64=0.1,
    verbose::Bool=true
)::Vector{SVector{3,Float64}}
    println("Initial number of base points: ", length(base_points))

    # Transform base points from [-1,1]^3 to actual domain
    domain_points = transform_points_to_domain(base_points, TR)
    println("After transformation: ", length(domain_points))

    # Initialize result vector with the base points
    all_points = Set{SVector{3,Float64}}()
    for point in domain_points
        push!(all_points, point)
    end
    println("After adding base points: ", length(all_points))

    # Create refined grids around each critical point
    for (i, row) in enumerate(eachrow(df_cheb))
        center_point = [row.x1, row.x2, row.x3]
        grid_points = create_refined_grid(center_point, TR, grid_size, N_sub)

        before_count = length(all_points)
        union!(all_points, grid_points)

        if verbose
            points_added = length(all_points) - before_count
            println("Critical point $i: Added $points_added new points")
        end
    end

    println("Total points after augmentation: ", length(all_points))
    return collect(all_points)
end

function create_refined_grid(
    center::Vector{Float64},
    TR,
    grid_size::Float64,
    N_sub::Int
)::Set{SVector{3,Float64}}
    # Calculate grid bounds
    half_size = (TR.sample_range * grid_size) / 2

    # Create ranges for each dimension
    ranges = [
        range(center[i] - half_size, center[i] + half_size, length=N_sub)
        for i in 1:3
    ]

    # Generate grid points
    grid_points = Set{SVector{3,Float64}}()
    for x in ranges[1], y in ranges[2], z in ranges[3]
        push!(grid_points, SVector{3,Float64}(x, y, z))
    end

    return grid_points
end

function transform_points_to_domain(
    points::Vector{SVector{3,Float64}},
    TR
)::Vector{SVector{3,Float64}}
    return map(points) do point
        SVector{3,Float64}(
            TR.sample_range / 2 * point[1] + TR.center[1],
            TR.sample_range / 2 * point[2] + TR.center[2],
            TR.sample_range / 2 * point[3] + TR.center[3]
        )
    end
end

transform_points_to_domain (generic function with 1 method)

In [68]:
function create_augmented_samples(
    base_points::Vector{SVector{3,Float64}},
    df_cheb::DataFrame,
    TR;
    N_sub::Int=5,
    verbose::Bool=true
)::Vector{SVector{3,Float64}}
    println("Initial number of base points: ", length(base_points))

    # First transform base points from [-1,1]^3 to actual domain
    domain_points = transform_points_to_domain(base_points, TR)
    println("After transformation: ", length(domain_points))

    # Instead of using a Set, let's keep track of points using a Vector
    # and implement our own duplicate checking with a tolerance
    augmented_points = Vector{SVector{3,Float64}}()
    sizehint!(augmented_points, length(domain_points))

    # Add transformed base points
    for point in domain_points
        push!(augmented_points, point)
    end
    println("After adding base points: ", length(augmented_points))

    # Add additional samples around critical points
    total_new_points = 0
    for (i, row) in enumerate(eachrow(df_cheb))
        point = [row.x1, row.x2, row.x3]
        local_samples = augment_samples_around_point(point, TR, N_sub)

        before_count = length(augmented_points)
        for sample in local_samples
            push!(augmented_points, sample)
        end

        points_added = length(augmented_points) - before_count
        total_new_points += points_added

        if verbose
            println("Critical point $i: Added $points_added new points")
        end
    end

    println("Total points after augmentation: ", length(augmented_points))

    # Return unique points with tolerance
    unique_tol = 1e-12  # Adjust this tolerance if needed
    unique_points = unique(p -> round.(p, digits=12), augmented_points)

    println("Final number of unique points: ", length(unique_points))

    return unique_points
end

function transform_points_to_domain(
    points::Vector{SVector{3,Float64}},
    TR
)::Vector{SVector{3,Float64}}
    return map(points) do point
        SVector{3,Float64}(
            TR.sample_range / 2 * point[1] + TR.center[1],
            TR.sample_range / 2 * point[2] + TR.center[2],
            TR.sample_range / 2 * point[3] + TR.center[3]
        )
    end
end

function augment_samples_around_point(
    point::Vector{Float64},
    TR,
    N_sub::Int=5;
    cube_side_ratio::Float64=0.1
)::Vector{SVector{3,Float64}}
    side_length = TR.sample_range * cube_side_ratio
    half_side = side_length / 2

    x_coords = range(point[1] - half_side, point[1] + half_side, length=N_sub)
    y_coords = range(point[2] - half_side, point[2] + half_side, length=N_sub)
    z_coords = range(point[3] - half_side, point[3] + half_side, length=N_sub)

    return vec([SVector{3,Float64}(x, y, z)
                for x in x_coords,
                y in y_coords,
                z in z_coords])
end

augment_samples_around_point (generic function with 2 methods)

## Function 1: add samples around one point

In [41]:
# function augment_grid_around_point(
#     point::Vector{Float64},
#     TR,
#     N_sub::Int=5;
#     cube_side_ratio::Float64=0.1,
#     basis::Symbol=:legendre  # Changed default to legendre
# )::Array{SVector{3,Float64},3}

#     sample_range = TR.sample_range
#     side_length = sample_range * cube_side_ratio
#     half_side = side_length / 2

#     # Using legendre (equally spaced) points
#     x_coords = range(point[1] - half_side, point[1] + half_side, length=N_sub)
#     y_coords = range(point[2] - half_side, point[2] + half_side, length=N_sub)
#     z_coords = range(point[3] - half_side, point[3] + half_side, length=N_sub)

#     [SVector{3,Float64}(x, y, z)
#      for x in x_coords,
#      y in y_coords,
#      z in z_coords]
# end

# function augment_samples_around_point(
#     point::Vector{Float64},
#     TR,
#     N_sub::Int=5;
#     cube_side_ratio::Float64=0.1
# )::Vector{SVector{3,Float64}}
#     # TR.sample_range might be 2.0 if the original domain is [-1,1]^3
#     # Ensure we're working in the correct scale

#     # First, check if point is already in [-1,1]^3 domain
#     # If so, we need to scale it to the actual domain
#     if all(-1 ≤ x ≤ 1 for x in point)
#         # Transform from [-1,1]^3 to actual domain
#         point = TR.sample_range / 2 .* point .+ TR.center
#     end

#     # Now compute the local sampling region in the actual domain
#     side_length = TR.sample_range * cube_side_ratio
#     half_side = side_length / 2

#     # Create local sampling points in the actual domain
#     x_coords = range(point[1] - half_side, point[1] + half_side, length=N_sub)
#     y_coords = range(point[2] - half_side, point[2] + half_side, length=N_sub)
#     z_coords = range(point[3] - half_side, point[3] + half_side, length=N_sub)

#     # Create points and transform them back to [-1,1]^3 domain
#     points = Vector{SVector{3,Float64}}()
#     for x in x_coords, y in y_coords, z in z_coords
#         # Transform point back to [-1,1]^3 domain
#         scaled_point = 2 .* ([x, y, z] .- TR.center) ./ TR.sample_range
#         push!(points, SVector{3,Float64}(scaled_point))
#     end

#     return points
# end

function augment_samples_around_point(
    point::Vector{Float64},
    TR,
    N_sub::Int=5;
    cube_side_ratio::Float64=0.1
)::Vector{SVector{3,Float64}}
    # point is in the actual domain

    # Calculate side length in the actual domain
    domain_size = TR.sample_range  # This is the full domain size
    side_length = domain_size * cube_side_ratio
    half_side = side_length / 2

    # Create local sampling points in the actual domain
    x_coords = range(point[1] - half_side, point[1] + half_side, length=N_sub)
    y_coords = range(point[2] - half_side, point[2] + half_side, length=N_sub)
    z_coords = range(point[3] - half_side, point[3] + half_side, length=N_sub)

    # Transform points to [-1,1]^3 domain
    points = Vector{SVector{3,Float64}}()
    for x in x_coords, y in y_coords, z in z_coords
        # Transform from actual domain to [-1,1]^3
        scaled_point = @. 2 * ([x, y, z] - TR.center) / TR.sample_range
        push!(points, SVector{3,Float64}(scaled_point))
    end

    return points
end

augment_samples_around_point (generic function with 2 methods)

# Function 2: Create augmented grid using DataFrame points

OLd code that seems broken.

In [43]:
# function create_augmented_grid(
#     base_grid::Array{SVector{3,Float64},3},
#     df_cheb::DataFrame;
#     N_sub::Int=5,
#     basis::Symbol=:legendre
# )::Array{SVector{3,Float64},3}

#     # Calculate new grid size
#     current_size = size(base_grid, 1)  # Should be 40
#     points_to_add = nrow(df_cheb) * N_sub^3
#     new_side_length = ceil(Int, ∛(current_size^3 + points_to_add))

#     # Create new grid array
#     new_grid = Array{SVector{3,Float64},3}(undef, new_side_length, new_side_length, new_side_length)

#     # Copy base grid
#     for i in 1:size(base_grid, 1), j in 1:size(base_grid, 2), k in 1:size(base_grid, 3)
#         new_grid[i, j, k] = base_grid[i, j, k]
#     end

#     # Add sub-sampled points in the remaining space
#     current_idx = size(base_grid, 1) * size(base_grid, 2) * size(base_grid, 3)

#     for row in eachrow(df_cheb)
#         point = [row.x1, row.x2, row.x3]
#         sub_grid = augment_grid_around_point(point, TR, N_sub; basis=basis)

#         for point in vec(sub_grid)
#             idx_i = (current_idx ÷ new_side_length^2) + 1
#             idx_j = ((current_idx % new_side_length^2) ÷ new_side_length) + 1
#             idx_k = (current_idx % new_side_length) + 1

#             if idx_i <= new_side_length && idx_j <= new_side_length && idx_k <= new_side_length
#                 new_grid[idx_i, idx_j, idx_k] = point
#                 current_idx += 1
#             end
#         end
#     end

#     return new_grid
# end


function create_augmented_samples(
    base_points::Vector{SVector{3,Float64}},
    df_cheb::DataFrame,
    TR;
    N_sub::Int=5,
    verbose::Bool=false
)::Vector{SVector{3,Float64}}
    # Initialize with base points (already in [-1,1]^3)
    augmented_points = Set{SVector{3,Float64}}()

    # Add base points
    for point in base_points
        push!(augmented_points, point)
    end

    if verbose
        println("Base points: $(length(augmented_points))")
    end

    # Add additional samples around critical points
    for (i, row) in enumerate(eachrow(df_cheb))
        # Critical points are in the actual domain
        point = [row.x1, row.x2, row.x3]
        local_samples = augment_samples_around_point(point, TR, N_sub)

        before_count = length(augmented_points)
        for sample in local_samples
            push!(augmented_points, sample)
        end

        if verbose
            println("Added $(length(augmented_points) - before_count) points around critical point $i")
        end
    end

    if verbose
        println("Total points: $(length(augmented_points))")
    end

    return collect(augmented_points)
end

create_augmented_samples (generic function with 1 method)

In [37]:
# function create_augmented_samples(
#     base_points::Vector{SVector{3,Float64}},
#     df_cheb::DataFrame,
#     TR;
#     N_sub::Int=5,
#     verbose::Bool=true
# )::Vector{SVector{3,Float64}}
#     # Initialize with base points
#     augmented_points = Set{SVector{3,Float64}}()

#     # Add base points (assumed to be already in correct scale)
#     initial_count = length(base_points)
#     for point in base_points
#         push!(augmented_points, point)
#     end
#     after_base_count = length(augmented_points)

#     if verbose
#         println("Initial points: $initial_count")
#         println("After adding base points: $after_base_count")
#         println("Duplicates removed: $(initial_count - after_base_count)")
#     end

#     # Add additional samples around critical points
#     total_new_points = 0
#     for (i, row) in enumerate(eachrow(df_cheb))
#         point = [row.x1, row.x2, row.x3]
#         local_samples = augment_samples_around_point(point, TR, N_sub)
#         before_count = length(augmented_points)
#         for sample in local_samples
#             push!(augmented_points, sample)
#         end
#         points_added = length(augmented_points) - before_count
#         total_new_points += points_added

#         if verbose
#             println("Point $i: Added $points_added new points out of $(length(local_samples)) samples")
#         end
#     end

#     if verbose
#         println("Total new points added: $total_new_points")
#         println("Final total: $(length(augmented_points))")
#     end

#     return collect(augmented_points)
# end

create_augmented_samples (generic function with 1 method)

In [45]:
# Create base grid with 40 points per dimension
GN = 50  # This will give us 40 points per dimension (GN + 1)
# base_grid = generate_grid_small_n(3, GN, basis=:legendre)  # 40×40×40 grid
augmented_grid = create_augmented_grid(grid, df_cheb)
println(size(grid))
println(size(augmented_grid))
points = vec(grid)
augmented_samples = create_augmented_samples(points, df_cheb, TR)
println(size(points))
println(size(augmented_samples))

(101, 101, 101)
(102, 102, 102)
(1030301,)
(1034551,)


In [46]:

GLMakie.activate!()
fig = create_interactive_level_set_view(
    f=f,
    points=augmented_samples,
    df=df_cheb,  # Your DataFrame with critical points
    z_range=(-3.0, 2.0),  # Min and max levels to show
    params=VisualizationParameters(
        point_tolerance=0.1,  # How close points need to be to the level
        fig_size=(1000, 800)  # Size of the visualization window
    )
)
display(fig)

GLMakie.Screen(...)

In [ ]:
augmented_samples = create_augmented_samples(points, df_cheb, TR, verbose=true)

Initial points: 1030301
After adding base points: 1030301
Duplicates removed: 0
Point 1: Added 125 new points out of 125 samples
Point 2: Added 125 new points out of 125 samples
Point 3: Added 125 new points out of 125 samples
Point 4: Added 125 new points out of 125 samples
Point 5: Added 125 new points out of 125 samples
Point 6: Added 125 new points out of 125 samples
Point 7: Added 125 new points out of 125 samples
Point 8: Added 125 new points out of 125 samples
Point 9: Added 125 new points out of 125 samples
Point 10: Added 125 new points out of 125 samples
Point 11: Added 125 new points out of 125 samples
Point 12: Added 125 new points out of 125 samples
Point 13: Added 125 new points out of 125 samples
Point 14: Added 125 new points out of 125 samples
Point 15: Added 125 new points out of 125 samples
Point 16: Added 125 new points out of 125 samples
Point 17: Added 125 new points out of 125 samples
Point 18: Added 125 new points out of 125 samples
Point 19: Added 125 new point

1034551-element Vector{SVector{3, Float64}}:
 [0.011181512098088597, -0.018587781625690167, -0.07871143220399279]
 [-0.05062020384744746, -0.111043255274069, -0.11986940938111329]
 [-0.0437581831539186, 0.05397940512091629, 0.0729279243696638]
 [0.0074603564917776815, -0.1196373652590755, 0.11479892628377036]
 [0.07871143220399279, 0.11986940938111329, -0.11824824183464175]
 [-0.08419042079817318, -0.11824824183464175, -0.09641963455664106]
 [-0.1196373652590755, -0.1196373652590755, -0.11998548759121794]
 [-0.0668622725634514, -0.08680904797766996, 0.036726870550546485]
 [-0.10453954897199405, 0.05397940512091629, -0.08934369300972181]
 [-0.08680904797766996, -0.047212030796010274, 0.1080003182482236]
 ⋮
 [0.06053794361383677, -0.11479892628377035, 0.11583020450321227]
 [-0.1175556969298898, -0.10632136313619775, -0.06053794361383676]
 [-0.1196373652590755, -0.022265730459539165, 0.11365658751732002]
 [0.06053794361383677, -0.029553468819840147, 0.09415131182956392]
 [0.11824824183464

In [40]:
# ts = vec(grid)

GLMakie.activate!()
fig_2 = create_interactive_level_set_view(
    f=f,
    points=augmented_samples,
    df=df_cheb,  # Your DataFrame with critical points
    z_range=(-3.0, 2.0),  # Min and max levels to show
    params=VisualizationParameters(
        point_tolerance=0.1,  # How close points need to be to the level
        fig_size=(1000, 800)  # Size of the visualization window
    )
)
display(fig)

GLMakie.Screen(...)

In [73]:
# function augment_samples_around_point(
#     point::Vector{Float64},
#     TR,
#     N_sub::Int=5;
#     cube_side_ratio::Float64=0.1
# )::Vector{SVector{3,Float64}}
#     sample_range = TR.sample_range
#     side_length = sample_range * cube_side_ratio
#     half_side = side_length / 2

#     # Create local sampling points
#     x_coords = range(point[1] - half_side, point[1] + half_side, length=N_sub)
#     y_coords = range(point[2] - half_side, point[2] + half_side, length=N_sub)
#     z_coords = range(point[3] - half_side, point[3] + half_side, length=N_sub)

#     # Return as a flat vector of points
#     return vec([SVector{3,Float64}(x, y, z) for x in x_coords, y in y_coords, z in z_coords])
# end

# function create_augmented_samples(
#     base_points::Vector{SVector{3,Float64}},
#     df_cheb::DataFrame;
#     N_sub::Int=5
# )::Vector{SVector{3,Float64}}
#     # Initialize with base points
#     augmented_points = Set{SVector{3,Float64}}()

#     # Add base points
#     for point in base_points
#         push!(augmented_points, point)
#     end

#     # Add additional samples around critical points
#     for row in eachrow(df_cheb)
#         point = [row.x1, row.x2, row.x3]
#         local_samples = augment_samples_around_point(point, TR, N_sub)
#         for sample in local_samples
#             push!(augmented_points, sample)
#         end
#     end

#     return collect(augmented_points)
# end

# # Usage example:
# # Convert your original grid to a vector if needed:
# # base_points = vec(base_grid)
# # augmented_points = create_augmented_samples(base_points, df_cheb)
# # println("Original points: $(length(base_points))")
# # println("Augmented points: $(length(augmented_points))")

create_augmented_samples (generic function with 1 method)

In [79]:
base_points = vec(base_grid)
augmented_points = create_augmented_samples(base_points, df_cheb)
println("Original points: $(length(base_points))")
println("Augmented points: $(length(augmented_points))") 

Original points: 132651
Augmented points: 136901


We are not correctly increasing the size of the sample grid. I thought we already had something efficient to augment the samples per level set, maybe the slider is much easier... In the end, I just want the Makie plot with the slider moving really slowly and a bit of rotation and output the result of that ---> 

In [83]:
create_level_set_visualization(f, grid, df_cheb, (-3.0, 6.))

In [84]:
create_level_set_visualization(f, augmented_grid, df_cheb, (-3.0, 6.0))

## Function 3: Produce Video 

In [40]:
function make_level_sets_video(
    f,
    grid::Array{SVector{3,T},3},
    z_range::Tuple{Real,Real},
    output_path::String;
    fps::Int=30,
    duration::Float64=20.0,
    tolerance::Float64=0.1
) where {T<:AbstractFloat}

    n_frames = round(Int, fps * duration)
    level_sequence = range(z_range[1], z_range[2], length=n_frames)

    fig = Figure()
    ax = Axis3(fig[1, 1],
        title="Level Set Animation",
        xlabel="x₁",
        ylabel="x₂",
        zlabel="x₃")

    grid_points = vec(grid)
    x_range = extrema(p[1] for p in grid_points)
    y_range = extrema(p[2] for p in grid_points)
    z_range_grid = extrema(p[3] for p in grid_points)

    limits!(ax, x_range..., y_range..., z_range_grid...)

    values = reshape(map(f, grid_points), size(grid))
    level_points = Observable(Point3f[])
    point_alphas = Observable(Float32[])

    scatter!(ax, level_points,
        color=:blue,
        markersize=0.8,
        alpha=point_alphas)  # Changed from opacity to alpha

    function update_visualization(level::T) where {T<:AbstractFloat}
        all_points = Point3f[]
        all_alphas = Float32[]

        for (point, value) in zip(grid_points, vec(values))
            dist_from_level = abs(value - level)
            if dist_from_level ≤ tolerance
                alpha = 1.0 - (dist_from_level / tolerance)
                push!(all_points, Point3f(point...))
                push!(all_alphas, Float32(alpha))
            end
        end

        level_points[] = all_points
        point_alphas[] = all_alphas
    end

    record(fig, output_path, 1:n_frames; framerate=fps) do frame
        current_level = level_sequence[frame]
        update_visualization(current_level)

        ax.azimuth[] = 1.7π + 0.4 * sin(2π * frame / n_frames)
        ax.elevation[] = π / 4 + 0.3 * cos(2π * frame / n_frames)
    end
end

# Try it:
z_range = (-3.0, 6.0)
make_level_sets_video(tref_3d, augmented_grid, z_range, "level_sets_video.mp4"; tolerance=0.1)

MethodError: MethodError: no method matching isless(::Vector{Float32}, ::Float64)
The function `isless` exists, but no method is defined for this combination of argument types.

Closest candidates are:
  isless(!Matched::Missing, ::Any)
   @ Base missing.jl:87
  isless(::Any, !Matched::Missing)
   @ Base missing.jl:88
  isless(!Matched::Union{Arblib.Arf, Arblib.ArfRef}, ::Union{Float16, Float32, Float64})
   @ Arblib ~/.julia/packages/Arblib/G8AGm/src/predicates.jl:143
  ...


In [37]:
z_range = (-3.0, 6.0)
video_level_sets(tref_3d, augmented_grid, z_range, "level_sets_video.mp4"; tolerance=0.1)


MethodError: MethodError: no method matching video_level_sets(::typeof(tref_3d), ::Array{SVector{3, Float64}, 3}, ::Tuple{Float64, Float64}, ::String; tolerance::Float64)
This error has been manually thrown, explicitly, so the method may exist but be intentionally marked as unimplemented.

Closest candidates are:
  video_level_sets(::Any, ::Array{SVector{3, Float64}, 3}, ::Tuple{Real, Real}, ::String; fps, duration) got unsupported keyword argument "tolerance"
   @ Main ~/globtim/Examples/Notebooks/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X25sZmlsZQ==.jl:2
  video_level_sets(::Any, ::Array{SVector{3, T}, 3}, ::Tuple{Real, Real}, ::String; fps, duration, tolerance) where T<:AbstractFloat
   @ Main ~/globtim/Examples/Notebooks/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X25sZmlsZQ==.jl:1


We look at the level of the grid, we just sample more densely in `x, y, z` space around each of the critical points from the dataframe ? --> we reuse the grid to generate all the level sets. 

In [13]:
# GLMakie.closeall()